In [1]:
import csv
from collections import Counter
import collections
import numpy as np
import operator
import h5py
import argparse
import sys
import re
import codecs
import string
import itertools


In [2]:
FILE_PATHS = {"full": ("data/train_chars.txt",
                      "data/valid_chars.txt",
                      "data/valid_chars_kaggle_answer.txt",
                      "data/valid_chars_kaggle.txt",
                      "data/test_chars.txt",)}

train_path, valid_path, valid_kaggle_answer_path, valid_kaggle_path, test_path = FILE_PATHS["full"]

In [3]:
train_X = (list(csv.reader(open(train_path, 'rb'), delimiter=' ')))[0]
valid_reduced_X = list(csv.reader(open(valid_path, 'rb'), delimiter=' '))[0]
valid_kaggle  = list(csv.reader(open(valid_kaggle_path, 'rb'), delimiter=' '))
valid_answers = (list(csv.reader(open(valid_kaggle_answer_path, 'rb'), delimiter=',')))[1:]
test = list(csv.reader(open(test_path, 'rb'), delimiter=' '))

char_dict = dict.fromkeys(string.ascii_lowercase, 0)
count = 1
for key in char_dict.keys():
    char_dict[key] = count
    count = count + 1

In [4]:


def get_outputs(X):
    #space = 2, non-space = 1
    outputs = np.zeros(len(X))
    for i in range(0, len(X)-1):
        if X[i+1] == char_dict['<space>']:
            outputs[i] = 2
        else:
            outputs[i] = 1
    outputs[-1] = 1
    return outputs

for i in range(0,len(train_X)):
    char = train_X[i]
    if char not in char_dict.keys():
        char_dict[char] = count
        count = count + 1
    train_X[i] = char_dict[train_X[i]]
    
for i in range(0,len(valid_reduced_X)):
    char = valid_reduced_X[i]
    if char not in char_dict.keys():
        char_dict[char] = count
        count = count + 1
    valid_reduced_X[i] = char_dict[valid_reduced_X[i]]
    
for i in range(0,len(valid_kaggle)):
    for j in range(0,len(valid_kaggle[i])):
        char = valid_kaggle[i][j]
        if char not in char_dict.keys():
            char_dict[char] = count
            count = count + 1
        valid_kaggle[i][j] = char_dict[valid_kaggle[i][j]]
        
for i in range(0,len(test)):
    for j in range(0,len(test[i])):
        char = test[i][j]
        if char not in char_dict.keys():
            char_dict[char] = count
            count = count + 1
        test[i][j] = char_dict[test[i][j]]

In [5]:
#backprop length and batch sizes
l = 32 #backprop length
b = 5  #batch size

n = len(train_X)
padding = np.ones((l*b - (n % (l*b)))%(l*b))
padded_train_X = np.append(train_X,padding)
padded_train_Y = get_outputs(padded_train_X)
padded_n = len(padded_train_X)
train_rnn_X = np.zeros((padded_n/(b*l), b, l))
train_rnn_Y = np.zeros((padded_n/(b*l), b, l))
for j in range(0,len(train_rnn_X[0])):
    for i in range(0,len(train_rnn_X)):
        segment_X = padded_train_X[l*len(train_rnn_X)*j + l*i:l*len(train_rnn_X)*j + l*i + l]
        segment_Y = padded_train_Y[l*len(train_rnn_X)*j + l*i:l*len(train_rnn_X)*j + l*i + l]
        train_rnn_X[i][j] = segment_X
        train_rnn_Y[i][j] = segment_Y
        
valid_reduced_Y = get_outputs(valid_reduced_X)
train_Y = get_outputs(train_X)

length = len(sorted(test,key=len, reverse=True)[0])
length = max(len(sorted(valid_kaggle ,key=len, reverse=True)[0]), length)
test = np.array([xi+[char_dict['</s>']]*(length-len(xi)) for xi in test])
valid_kaggle = np.array([xi+[char_dict['</s>']]*(length-len(xi)) for xi in valid_kaggle])

In [6]:
filename = 'data' + '.hdf5'
with h5py.File(filename, "w") as f:
    f['train_rnn_X'] = train_rnn_X
    f['train_rnn_Y'] = train_rnn_Y
    f['train_X'] = train_X
    f['train_Y'] = train_Y
    f['valid_reduced_X'] = valid_reduced_X
    f['valid_reduced_Y'] = valid_reduced_Y
    f['test'] = test
    f['valid_kaggle'] = valid_kaggle
    f['valid_answers'] = (np.array(valid_answers).astype(int))[:,1]


In [ ]:
valid_kaggle.shape

In [ ]:
test = np.array([np.array(xi) for xi in a])

In [ ]:
len(sorted(valid_kaggle ,key=len, reverse=True)[0])

In [ ]:
length